In [ ]:
from copy import copy

RITUAL = 3
CULTIST_HP = 48

class Game:
    def __init__(self):
        self.turn = 1
        self.todraw = 5
        
        self.cultisthp = 48
        
        self.hplost = 0
        self.energy = 3
        self.block = 0
        self.deck = tuple(sorted(['strike']*5 + ['block']*5))
        self.hand = tuple()
        self.discard = tuple()
        
    def __hash__(self):
        assert self.deck == tuple(sorted(self.deck))
        assert self.hand == tuple(sorted(self.hand))
        if self.discard != tuple(sorted(self.discard)):
            print(self.discard)
            print(tuple(sorted(self.discard)))
            assert self.discard == tuple(sorted(self.discard)) 
        
        return hash((
            self.turn, self.cultisthp, self.hplost, self.energy, self.block,
            self.deck, self.hand, self.discard
        ))
    
    def __str__(self):
        return str(self.__dict__)
    

# f(gamedetails) -> tuple<gamedetails>, reason string
def FAN_OUT(game):
    if game.todraw:
        self = copy(game)
        self.todraw -= 1
        if not self.deck:
            self.deck = self.discard
            self.discard = tuple()
        return tuple(
            move_deck_to_hand(copy(self), x)
            for x in range(0, len(self.deck))
        ), 'draw'
    
    if game.energy == 0:
        return (endturn(copy(game)),), 'end'
    
    return tuple(
        playcard(copy(game), cardname)
        for cardname in game.hand
    ), 'choice'


# below are actions
#   f(gamestate, *params) -> modified gamestate
def move_deck_to_hand(new, x):
    new.hand = tuple(sorted(new.hand + (new.deck[x],)))
    new.deck = tuple(new.deck[:x]) + tuple(new.deck[x+1:])
    return new

def endturn(new):
    new.discard = tuple(sorted(new.discard + new.hand))
    new.hand = tuple()
    
    if new.turn > 1:
        damage = (new.turn-1)*RITUAL + 6
        blocked = min(damage, new.block)
        new.hplost += damage - blocked
    
    new.todraw = 5
    new.energy = 3
    new.block = 0
    new.turn += 1
    return new
    
def playcard(new, name):
    x = new.hand.index(name)
    new.hand = tuple(new.hand[:x]) + tuple(new.hand[x+1:])
    
    if name == 'strike':
        new.cultisthp -= 6
        new.energy -= 1
        
    if name == 'block':
        new.block += 5
        new.energy -= 1
        
    new.discard = tuple(sorted(new.discard + (name,)))
    return new


g = Game()
print(g)

for c in FAN_OUT(g):
    print(bin(hash(c)))

In [2]:
from collections import deque, defaultdict


# hash : gamedetails | (tuple<hash>, reason) | (lost?, int score)
TREE = {}  
_DETAILS = {}

# hash : status
QUEUE = deque()
PRESENT = set()

WINS = []
LOSSES = []

def process(gamehash):
    self = TREE[gamehash]
    assert type(self) == Game
    
    if self.cultisthp <= 0:
        WINS.append((self.turn, self.hplost))
        return 'win', self.hplost
    if self.hplost >= 20:
        LOSSES.append((self.turn, self.cultisthp))
        return 'lose', self.cultisthp
    
    ch, reason = FAN_OUT(self)
    children = {hash(c):c for c in ch}
    for key, child in children.items():
        TREE[key] = child
        if key in _DETAILS:
            assert str(_DETAILS[key]) == str(child)
        else:
            _DETAILS[key] = child
        if key not in PRESENT:
            PRESENT.add(key)
            QUEUE.appendleft(key)
        
    return tuple(children.keys()), reason


root = Game()
_DETAILS[hash(root)] = root
TREE[hash(root)] = root
TREE[hash(root)] = process(hash(root))

while QUEUE:
    key = QUEUE.pop()
    PRESENT.remove(key)
    TREE[key] = process(key)

# TREE, QUEUE
print(f'{len(TREE)/1000}k nodes explored')
len(WINS), len(LOSSES)

8.49k nodes explored


(432, 232)

In [4]:
# the big cleanup
"""
test(node) ->
    choice: [
        {win: 0.75, loss: 0.25},
        {win: 0.65, loss: 0.35},
        ]

convert a 
    {} hash -> (tuple<hash>, reason string) | (lost?, score number)
to
    {} hash -> {win: ??, loss: ??}
"""

LINKS_DOWN = {}
PROBS_DOWN = {}
LINKS_UP = {}

def persist(key, values, probs):
    LINKS_DOWN[key] = values
    PROBS_DOWN[key] = probs
    
    for child in values:
        if child not in LINKS_UP:
            LINKS_UP[child] = []
        LINKS_UP[child].append(key)


def squash_r_block(rsh):
    """
    returns {cblock-hash : prob}
    """
    children, reason = TREE[rsh]
    assert reason != 'choice'
    
    if children == 'win':
        return {}
    if children == 'lose':
        return {}
    
    outcomes = {}
    p_each = 1/len(children)
    for child_hash in children:
        _, reason = TREE[child_hash]
        if reason == 'choice':
            mix = {child_hash : 1}
        else:
            mix = squash_r_block(child_hash)

        for k,v in mix.items():
            outcomes[k] = outcomes.get(k, 0) + v*p_each
    
    persist(rsh, tuple(outcomes.keys()), tuple(outcomes.values()))
    return outcomes
    
def _squash_c_block_array(rsh):
    """
    returns {}rblock-hash
    """
    children, reason = TREE[rsh]
    assert reason == 'choice'
    assert type(children) == tuple
    
    outcomes = []
    for child_hash in children:
        _, reason = TREE[child_hash]
        if reason == 'choice':
            outcomes.extend(_squash_c_block_array(child_hash))
        else:
            outcomes.append(child_hash) 
            
    return outcomes

def squash_c_block(rsh):
    r = frozenset(_squash_c_block_array(rsh))
    persist(rsh, r, [1/len(r) for _ in r])
    return r

# -- dummy THICC recursive bit
def step_r(rkey):
    for ckey in squash_r_block(rkey):
        step_c(ckey)
        
def step_c(ckey):
    for rkey in squash_c_block(ckey):
        step_r(rkey)
    
step_r(hash(root))

In [5]:
len(LINKS_DOWN), len(LINKS_UP), len(TREE)

(5427, 2235, 8490)

In [ ]:
## interactive

current = hash(root)
while True:
    print(_DETAILS[current])
    for n,p in zip(LINKS_DOWN[current], PROBS_DOWN[current]):
        print(f'  {p:.5f} {n}')
        
    c = input()
    if c == 'q':
        break
        
    current = tuple(LINKS_DOWN[current])[int(c)]
    
    if TREE[current] in ('win', 'lose'):
        print(TREE[current])
        break

In [ ]:
# -- and now make these functions useful

BEST = {}
FLOW = {}

root = tuple(TREE.keys())[0]

def chunk_step_r(rkey):
    keys = squash_r_block(key)
    paths = tuple(keys.keys())
    results = 
    probs = tuple(keys.values())
        
def chunk_step_c(ckey):
    
chunk_step(root)

In [ ]:
    
    
#adfadfsdfad
#     if ckey in BEST:
#         return BEST[ckey]
    
#     keys = squash_c_block(ckey)
#     options = zip(keys, (avg_outcome(k) for k in keys))
    
#     try:
#         bestkey, bestscore = max(options, key=lambda t: (t[1]['win'], t[0]))
#     except TypeError:
#         print(tuple(zip(keys, (squash_r_block(k) for k in keys))))
#         raise TypeError
    
#     FLOW[ckey] = bestkey
#     BEST[ckey] = bestscore
    
#     return bestscore
    
# def avg_outcome(rkey):
#     if rkey in BEST:
#         return BEST[rkey]
    
#     keys = squash_r_block(rkey)
    
#     outcomes = {}
#     for k,p in keys.items():
#         for condition, value in best_choice(k).items():
#             outcomes[condition] = outcomes.get(condition, 0) + value/len(keys)
    
#     BEST[rkey] = outcomes
#     return outcomes
        

# best_choice(4230168099146957074)
chunk_step(8837678220020631969)

In [ ]:
root = tuple(TREE.keys())[0]
squash_r_block(root)

In [ ]:
squash_c_block(5238642625037216471)

In [ ]:
squash_r_block(-7149946983080482141)

In [ ]:
squash_c_block(6160438167907557174)

In [ ]:
squash_r_block(-7515693022861666502)

In [ ]:
TREE[-7515693022861666502]

In [ ]:
TREE[1527140810226612874]

In [ ]:
class Status:
    def __init__(self):
        self.win = 0.0
        self.loss = 0.0
        
    def __hash__(self):
        return hash((self.win, self.loss))
    
    def __str__(self):
        return f'{self.win}/{self.win+self.loss}'

def probpath(h):
    if h in PROBS:
        return PROBS[h]
    
    children, reason = TREE[h]
    if type(children) != tuple:
        result = ['win', 'loss'][children]
        status = Status()
        if result == 'win':
            status.win = 1.0
        elif result == 'loss':
            status.loss = 1.0
        return status
        
    if reason == 'choice':
        return tuple(probpath(c) for c in children)
    
    local_probs = Status()
    p_each = 1/len(children)
    for child in children:
        results = probpath(child)
        
        if type(results) == Status:
            local_probs.win += results.win*p_each
            local_probs.loss += results.loss*p_each
            
        elif type(results) == tuple:
            local_probs[results] = p_each
                
    return local_probs

probpath(tuple(TREE.keys())[-500])

In [ ]:
def seepath()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

loss = defaultdict(lambda: [])
for t, hploss in WINS:
    loss[t].append(hploss)
    
sns.distplot(loss[4])


In [ ]:
sorted(LOSSES)

In [ ]:
for i in range(3, 6):
    sns.distplot(loss[i], norm_hist=False, kde=False, bins=range(0, 20, 1))

plt.show()

In [ ]:
help(sns.distplot)